In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data from output.csv
data = pd.read_csv('../output/output_binary.csv')

# Split features and labels
feature_df = data.drop(data.columns[:2], axis=1)
X = np.asarray(feature_df)
y = np.asarray(data['pii_exist'])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))
y_train_one_hot = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_one_hot = tf.one_hot(y_test, depth=num_classes).numpy()

# Define function to create model
def create_model(num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create a list of models representing each client
num_clients = 10
client_models = [create_model(num_classes) for _ in range(num_clients)]

# Define a function for training on each client's data
def train_on_client(X, y, model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)
    return model

# Train each client's model
for i, model in enumerate(client_models):
    X_client_train, _, y_client_train, _ = train_test_split(X_train, y_train_one_hot, test_size=0.8, random_state=i)  # Use a portion of the training data for each client
    client_models[i] = train_on_client(X_client_train, y_client_train, model)

# Aggregate weights of all client models to update the global model
global_model = create_model(num_classes)
for model in client_models:
    global_model.set_weights([(w1 + w2) / 2 for w1, w2 in zip(global_model.get_weights(), model.get_weights())])
    global_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    y_pred = np.argmax(global_model.predict(X_test), axis=1)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1 Score:", f1)

# Compile the global model
global_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Predict classes using the global model
y_pred = np.argmax(global_model.predict(X_test), axis=1)

# Calculate confusion matrix and F1 score
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print("F1 Score:", f1)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
F1 Score: 0.6364375993339892
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
F1 Score: 0.9436459700394445
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9568942684829826
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
F1 Score: 0.959419652956359
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
F1 Score: 0.956775493947521
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
F1 Score: 0.956775493947521
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9620703881753607
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
F1 Score: 0.9620703881753607
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9515060520433554
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
F1 Score: 0.959419652956359
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Confusion Matrix:
[[256  14]
 [  1  91]]
F1 Score: 0.959419652956359


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data from output.csv
data = pd.read_csv('../output/output.csv')

# Split features and labels
feature_df = data.drop(data.columns[:2], axis=1)
X = np.asarray(feature_df)
y = np.asarray(data['pii_exist'])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))
y_train_one_hot = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_one_hot = tf.one_hot(y_test, depth=num_classes).numpy()

# Define SVM model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_hinge',
              metrics=['accuracy'])

# Define privacy parameters
epsilon = 20 # Privacy budget
sensitivity = 1  # Sensitivity of the gradients

# Define a function for training on each client's data with differential privacy
def train_on_client_dp(X, y, model, epsilon):
    model.compile(optimizer='adam',
                  loss='categorical_hinge',
                  metrics=['accuracy'])
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)

    # Add noise to the gradients
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            for weight in layer.trainable_variables:
                noise = tf.random.normal(shape=weight.shape, stddev=sensitivity / epsilon)
                weight.assign_add(noise)

    return model

# Federated learning loop with differential privacy
client_models = []  # List to store models of individual clients
for i in range(10):  # 10 clients
    # Divide the training data into 10 parts
    start_index = int(i * len(X_train) / 10)
    end_index = int((i + 1) * len(X_train) / 10)
    X_client_train = X_train[start_index:end_index]
    y_client_train = y_train_one_hot[start_index:end_index]

    # Train client model on its data with differential privacy
    client_model = train_on_client_dp(X_client_train, y_client_train, model, epsilon)
    client_models.append(client_model)  # Store the trained model for aggregation

    # Evaluate client model on the test set
    y_pred = np.argmax(client_model.predict(X_test), axis=1)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Client", i+1, "F1 Score:", f1)

# Aggregate weights of all client models to update the global model
global_model = tf.keras.models.clone_model(model)  # Create a copy of the original model
for client_model in client_models:
    for global_layer, client_layer in zip(global_model.layers, client_model.layers):
        global_layer_weights = global_layer.get_weights()
        client_layer_weights = client_layer.get_weights()
        aggregated_weights = [(w1 + w2) / 2 for w1, w2 in zip(global_layer_weights, client_layer_weights)]
        global_layer.set_weights(aggregated_weights)

# Predict classes using the global model
y_pred = np.argmax(global_model.predict(X_test), axis=1)

# Calculate confusion matrix and F1 score
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print("F1 Score:", f1)

52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Client 1 F1 Score: 0.8411420563362862
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Client 2 F1 Score: 0.8811417943116095
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Client 3 F1 Score: 0.905981662024451
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Client 4 F1 Score: 0.912776140881875
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Client 5 F1 Score: 0.9161811416845413
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Client 6 F1 Score: 0.9162984029251067
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Client 7 F1 Score: 0.9158287528870898
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Client 8 F1 Score: 0.91881712217981
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Client 9 F1 Score: 0.9176490232183685
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Client 10 F1 Score: 0.9164809242569268
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Confusion Matrix:
[[1118  124]
 [  19  392]]
F1 Score: 0.9164809242569268


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data from output.csv
data = pd.read_csv('../output/output_binary.csv')

# Split features and labels
feature_df = data.drop(data.columns[:2], axis=1)
X = np.asarray(feature_df)
y = np.asarray(data['pii_exist'])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))
y_train_one_hot = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_one_hot = tf.one_hot(y_test, depth=num_classes).numpy()

# Define privacy parameters
epsilon = 20  # Privacy budget
delta = 1e-5  # Desired overall privacy failure probability
sensitivity = 1  # Sensitivity of the gradients

# Define SVM model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_hinge',
              metrics=['accuracy'])

# Define a function for training on each client's data with differential privacy
def train_on_client_dp(X, y, model, epsilon, delta):
    # Compute the scale parameter for Gaussian noise
    delta_prime = delta / (2 * len(X) / 32)  # Assuming batch size of 32
    sigma = np.sqrt(2 * np.log(1.25 / delta_prime)) * sensitivity / epsilon

    model.compile(optimizer='adam',
                  loss='categorical_hinge',
                  metrics=['accuracy'])
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)

    # Add noise to the gradients
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            for weight in layer.trainable_variables:
                noise = tf.random.normal(shape=weight.shape, stddev=sigma)
                weight.assign_add(noise)

    return model

# Federated learning loop with differential privacy
client_models = []  # List to store models of individual clients
for i in range(10):  # 10 clients
    # Divide the training data into 10 parts
    start_index = int(i * len(X_train) / 10)
    end_index = int((i + 1) * len(X_train) / 10)
    X_client_train = X_train[start_index:end_index]
    y_client_train = y_train_one_hot[start_index:end_index]

    # Train client model on its data with differential privacy
    client_model = train_on_client_dp(X_client_train, y_client_train, model, epsilon, delta)
    client_models.append(client_model)  # Store the trained model for aggregation

    # Evaluate client model on the test set
    y_pred = np.argmax(client_model.predict(X_test), axis=1)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Client", i+1, "F1 Score:", f1)

# Aggregate weights of all client models to update the global model
global_model = tf.keras.models.clone_model(model)  # Create a copy of the original model
for client_model in client_models:
    for global_layer, client_layer in zip(global_model.layers, client_model.layers):
        global_layer_weights = global_layer.get_weights()
        client_layer_weights = client_layer.get_weights()
        aggregated_weights = [(w1 + w2) / 2 for w1, w2 in zip(global_layer_weights, client_layer_weights)]
        global_layer.set_weights(aggregated_weights)

# Predict classes using the global model
y_pred = np.argmax(global_model.predict(X_test), axis=1)

# Calculate confusion matrix and F1 score
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print("F1 Score:", f1)

52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Client 1 F1 Score: 0.6434201809259958
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Client 2 F1 Score: 0.7422240275925631
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Client 3 F1 Score: 0.7627921762181248
